In [1]:
import numpy as np
import pandas as pd
import librosa
import time
import matplotlib.pyplot as plt
import ESC3
import multiprocessing

import tensorflow as tf
from tensorflow.keras.utils import to_categorical, plot_model

from tensorflow.compat.v1.keras.backend import set_session, clear_session, get_session
import gc

## Fold Training

In [2]:
fold_train_loss = []
fold_train_accuracy = []
fold_valid_loss = []
fold_valid_accuracy = []
fold_test = []

In [3]:
# Reset Keras Session
def reset_keras():
    
    # Get and close session
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()
    
    # Garbage collector call
    gc.collect()
    
    # Init new session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.compat.v1.Session(config=config))

In [4]:
# Define training parameters
max_epochs = 500
lr = 0.001
batch_size = 64
aug_rate=0.15

# Loop over the functions
for i in range(1, 6):
    
    
    # load the fold data
    train_d, train_l, val_d, val_l, test_d, test_l = ESC3.Load_Segments('ESC50', i)
    
    # Generate training and validation dataset
    #training_dataset = ESC3.CreateTrainingSet50(train_d, train_l, name=f'train_F{i}', batch_size=batch_size)
    training_dataset = ESC3.CreateValidationSet(train_d, train_l, name=f'train_F{i}', batch_size=batch_size)
    validation_dataset = ESC3.CreateValidationSet(val_d, val_l, name=f'valid_F{i}', batch_size=batch_size)
    
    # Initialize the network
    net = ESC3.PiczakNet50([60, 41, 3])
    loss_f = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    opt = tf.keras.optimizers.Adam(lr=lr)
    net.compile(optimizer=opt, loss=loss_f, metrics=["accuracy"])
    
    print(f'\n#############################################################')
    print(f'######################## FOLD-{i} #############################')
    print(f'#############################################################')
    
    # Train the network
    epoch_loss, epoch_acc, epoch_vl, epoch_va = ESC3.train(net, max_epochs, training_dataset, validation_dataset, batch_size=batch_size, aug_rate=aug_rate, verbose=True)
    
    # Append lossed for confidence plot
    fold_train_loss.append(epoch_loss)
    fold_train_accuracy.append(epoch_acc)
    fold_valid_loss.append(epoch_vl)
    fold_valid_accuracy.append(epoch_va)
    
    reset_keras()


#############################################################
######################## FOLD-1 #############################
#############################################################
Epoch  1: 	 t-loss: 37.714409 	 t-acc: 0.093750 	 v-loss: 3.984796 	 v-acc: 0.025789 	 time: 4.901
Epoch  2: 	 t-loss: 3.887333 	 t-acc: 0.221875 	 v-loss: 4.313457 	 v-acc: 0.021086 	 time: 3.111
Epoch  3: 	 t-loss: 3.304750 	 t-acc: 0.096875 	 v-loss: 4.139375 	 v-acc: 0.021845 	 time: 3.100
Epoch  4: 	 t-loss: 2.360854 	 t-acc: 0.262500 	 v-loss: 4.317555 	 v-acc: 0.021845 	 time: 3.103
Epoch  5: 	 t-loss: 2.106009 	 t-acc: 0.259375 	 v-loss: 4.425918 	 v-acc: 0.021845 	 time: 3.101
Epoch  6: 	 t-loss: 1.975324 	 t-acc: 0.237500 	 v-loss: 5.013802 	 v-acc: 0.021845 	 time: 3.100
Epoch  7: 	 t-loss: 2.058517 	 t-acc: 0.162500 	 v-loss: 4.794616 	 v-acc: 0.021845 	 time: 3.111
Epoch  8: 	 t-loss: 1.998615 	 t-acc: 0.262500 	 v-loss: 4.804891 	 v-acc: 0.021845 	 time: 3.110
Epoch  9: 	 t-loss: 1.974377

KeyboardInterrupt: 

In [ ]:
# Convert lists to array
fold_train_loss = np.asarray(fold_train_loss)
fold_train_accuracy = np.asarray(fold_train_accuracy)
fold_valid_loss = np.asarray(fold_valid_loss)
fold_valid_accuracy = np.asarray(fold_valid_accuracy)

# Compute mean and confidence interval
m_tl = []
m_ta = []
m_vl = []
m_va = []

ci_tl = []
ci_ta = []
ci_vl = []
ci_va = []

for i in range(max_epochs):
    m_tl.append(np.mean(fold_train_loss[:, i]))
    m_ta.append(np.mean(fold_train_accuracy[:, i]))
    m_vl.append(np.mean(fold_valid_loss[:, i]))
    m_va.append(np.mean(fold_valid_accuracy[:, i]))
    
    ci_tl.append(1.96*np.std(fold_train_loss[:, i])/np.mean(fold_train_loss[:, i]))
    ci_ta.append(1.96*np.std(fold_train_accuracy[:, i])/np.mean(fold_train_accuracy[:, i]))
    ci_vl.append(1.96*np.std(fold_valid_loss[:, i])/np.mean(fold_valid_loss[:, i]))
    ci_va.append(1.96*np.std(fold_valid_accuracy[:, i])/np.mean(fold_valid_accuracy[:, i]))

m_tl = np.asarray(m_tl)
m_ta = np.asarray(m_ta)
m_vl = np.asarray(m_vl)
m_va = np.asarray(m_va)

ci_tl = np.asarray(ci_tl)
ci_ta = np.asarray(ci_ta)
ci_vl = np.asarray(ci_vl)
ci_va = np.asarray(ci_va)

In [ ]:
# Plot loss
fig, ax = plt.subplots()
ax.plot(m_tl, label='train loss')
ax.fill_between(np.arange(0, max_epochs), (m_tl-ci_tl), (m_tl+ci_tl), color='b', alpha=.1)
ax.plt(m_vl, label='validation loss')
ax.fill_between(np.arange(0, max_epochs), (m_vl-ci_vl), (m_vl+ci_vl), color='r', alpha=.1)
plt.grid()
plt.legend()
plt.title('Loss Plot')
plt.show()

In [ ]:
# Plot accuracy
fig, ax = plt.subplots()
ax.plot(m_ta, label='train accuracy')
ax.fill_between(np.arange(0, max_epochs), (m_ta-ci_ta), (m_ta+ci_ta), color='b', alpha=.1)
ax.plt(m_va, label='validation accuracy')
ax.fill_between(np.arange(0, max_epochs), (m_va-ci_va), (m_va+ci_va), color='r', alpha=.1)
plt.grid()
plt.legend()
plt.title('Accuracy Plot')
plt.show()